# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK
## Prerequisites
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

In [ ]:
! pip install azure-search-documents --pre
! pip install openai
! pip install python-dotenv

## Import required libraries and environment variables

In [6]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,
) 

  
# Configure environment variables  
load_dotenv() 
model_deployment_name = os.getenv("AZURE_OPENAI_MODEL_NAME")
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential(key)

ImportError: cannot import name 'Vector' from 'azure.search.documents.models' (e:\Projects\Python\lightfoot-streamlit\.venv\lib\site-packages\azure\search\documents\models\__init__.py)

In [ ]:
# removes unwanted characters from the txt file

import os
import codecs

# Path to the directory containing the text files
directory = "../web-crawler/txt/www.lightfoot.ca/"

# Iterate through the directory
for filename in os.listdir(directory):
    # Check if the file is a text file
    if filename.endswith(".txt"):
        # Open the file in read mode and read its contents
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
            contents = f.read()
        # Convert the encoding to utf-8
        contents = contents.replace('\n\n', ' ').replace('\n', ' ').replace('â€', '').replace('Â', '').replace('©', '').replace('*', '').replace('•', '').replace('*', '').replace('“', '').replace('”', '').replace('  ', ' ').replace("LYRICS/CHORDS", '').replace('    ', ' ')
        #contents = contents.encode('utf-8')
        # Open the file in write mode and write the utf-8 encoded contents
        with codecs.open(os.path.join(directory, filename), 'w', encoding='UTF-8') as f:
            f.write(contents)  

In [ ]:
import uuid
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
import pandas as pd

# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings

# get a UUID - URL safe, Base64
def get_a_uuid():
    return str(uuid.uuid4())

# method to get the token length with the encoding
tokenizer_name = tiktoken.get_encoding("cl100k_base")
tokenizer = tiktoken.get_encoding(tokenizer_name.name)

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000, # this depends on which model you might use, for example with the 16k GPT models setting this to 8k is reasonable and maybe higher
    chunk_overlap=100,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""],
)

#function to return the number of tokens in a string
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    token_integers = encoding.encode(string)
    num_tokens = len(token_integers)

    return num_tokens

def get_text_before_double_space(input_string):  
    return input_string.split("  ", 1)[0] 


## Create dataframe with embeddings from txt files in a directory
Read your txt files into a dataframe that can be used to load the index:

In [ ]:
# open and read all the txt files and put them into chuncks in a dataframe, this takes the contents of
# the file and splits based on the text splitter.  this needs to be split because of the embeddings
# columns will be title, tokens, content

directory = "../web-crawler/txt/www.lightfoot.ca/"
chunk = {}
txt = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), "r", encoding="utf-8") as f:
                text = f.read()
                texts = text_splitter.create_documents([text])
                for i in texts:
                        chunk = {
                                "id": get_a_uuid(),  # generate a random uuid for the document
                                "title": get_text_before_double_space(i.page_content),
                                #"title": filename[:-4],  # remove the .txt extension from the filename and use this as the title
                                "content": i.page_content,
                                "sourcefile": filename,
                                "content_tokens": num_tokens_from_string(i.page_content, "cl100k_base"),
                                "category": "Lightfoot",
                                "contentVector": generate_embeddings(i.page_content)
                                }
                        txt.append(chunk)

df = pd.DataFrame(txt)
df

In [ ]:
# write DataFrame to a csv file  
df['title'].to_csv('output.csv', index=False)

## Create your search index
Create your search index schema and vector search configuration:

In [ ]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="sourcefile", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="lightfoot-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="lightfoot-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="lightfoot-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="category")],
        prioritized_content_fields=[SemanticField(field_name="content")],
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


## Insert text and embeddings into vector store from data frame
Creates a list called sections from the dataframe to be loaded into the index:

In [ ]:
# populate a list with the data we will use to store in the index
import re

def create_sections(df):
    for index, row in df.iterrows():
        yield {
            "id": row["id"],
            "title": row["title"],
            "content": row["content"],
            "sourcefile": row["sourcefile"],
            "category": row["category"],
            "contentVector": row["contentVector"],
            "@search.action": "upload",
        }
        
sections = create_sections(df)

## Load the sections list into the index
Loops thru a list called sections and creates a document ofr each item in the index:

In [ ]:
def index_sections(sections):
    print(
        f"Indexing sections into search index '{index_name}'"
    )

    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

    i = 0
    batch = []
    for s in sections:
        batch.append(s)
        i += 1
        if i % 1000 == 0:
            results = search_client.upload_documents(documents=batch)
            succeeded = sum([1 for r in results if r.succeeded])
            print(f"\tIndexed {len(results)} sections, {succeeded} succeeded")
            batch = []

    if len(batch) > 0:
        results = search_client.upload_documents(documents=batch)
        succeeded = sum([1 for r in results if r.succeeded])
        print(f"\tIndexed {len(results)} sections, {succeeded} succeeded")
        
index_sections(sections)

## Perform a vector similarity search

In [ ]:
# Pure Vector Search
query = "legend lives on from the chippewah"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=1, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vectors= [vector],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


## Perform a Cross-Field Vector Search

In [ ]:
# Cross-Field Vector Search
query = "types of anxiety disorders"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector, title")  
  
results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["title", "summary", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Summary: {result['summary']}")  
    print(f"Category: {result['category']}\n")  


## Perform a Multi-Vector Search

In [ ]:
# Multi-Vector Search
query = "challenger sales model"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
vector1 = Vector(value=generate_embeddings(query), k=3, fields="summaryVector")  
vector2 = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  
  
results = search_client.search(  
    search_text=None,  
    vectors=[vector1, vector2],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


## Perform a Pure Vector Search with a filter

In [ ]:
# Pure Vector Search with Filter
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    filter="category eq 'Challenger Customer'",
    select=["title", "content", "category"]
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


## Perform a Hybrid Search

In [ ]:
# Hybrid Search
query = "dans records"  
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["title", "content", "category"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


## Perform a Semantic Hybrid Search

In [ ]:
# Semantic Hybrid Search
query = "types of anxiety disorders"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector = Vector(value=generate_embeddings(query), k=3, fields="summaryVector")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["title", "summary", "category"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='dsm-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Summary: {result['summary']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")
